In [45]:
import pandas as pd
import numpy as np
from itertools import permutations
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA, QuadraticDiscriminantAnalysis as QDA
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split,RandomizedSearchCV,GridSearchCV,cross_val_score
from sklearn.metrics import confusion_matrix,accuracy_score
import xgboost as xgb

import warnings
warnings.filterwarnings("ignore")

def CVError(model,X,Y,cv=5):
    return 1-cross_val_score(model,X,Y,cv=cv).mean()

# Part I. Classification on 20newsgroup Data

Preprocess data

In [4]:
doc=pd.read_table(r'../data/20newsgroup/documents.txt',header=None)
wordlist=pd.read_table(r'../data/20newsgroup/wordlist.txt',header=None)[0].tolist()
target=pd.read_table(r'../data/20newsgroup/newsgroups.txt',header=None)[0].to_numpy()

In [5]:
data=np.zeros((16242,100),dtype=int)
for index, row in doc.iterrows():
    i,j = row[0],row[1]
    data[i-1,j-1] = 1

In [6]:
datadf=pd.DataFrame(data,columns=wordlist)
datadf

,aids,baseball,bible,bmw,cancer,car,card,case,children,christian,...,university,version,video,vitamin,war,water,win,windows,won,world
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16237,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16238,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16239,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
16240,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [7]:
X,Xtest,Y,Ytest=train_test_split(datadf,target,test_size=0.1,random_state=5054)

## 1.Build a random forest for this dataset 
and report the 5-fold cross validation value of the misclassification error. 
Note that you need to train the model by yourself, i.e., how many predictors 
are chosen in each tree and how many trees are used. There is no benchmark. Stop tuning when 
you feel appropriate. Report the best CV error, the corresponding confusion matrix and tuning 
parameters. What are the ten most important keywords based on variable importance?

In [10]:
parameters = {'n_estimators': np.arange(
    20, 100, 10), 'max_features': np.arange(0, 10, 1)}
RF = RandomizedSearchCV(RandomForestClassifier(),
                        param_distributions=parameters, cv=5)
RF.fit(X, Y)
print(RF.best_params_)

{'n_estimators': 90, 'max_features': 2}


In [18]:
RF_best = RF.best_estimator_
RF_best.fit(X,Y)
top10=[datadf.columns.values[i] for i in (np.argsort(RF_best.feature_importances_)[-1:-11:-1])]
print("best CV error:",CVError(RF_best,X,Y,cv=5)) 
print("confusion matrix:\n",confusion_matrix(RF_best.predict(Xtest),Ytest))
print("tuning parameters:",RF.best_params_)
print("ten most important keywords:\n", top10)

best CV error: 0.18717899877054156
confusion matrix:
 [[416  32  50  24]
 [ 17 262  21  17]
 [ 20  13 148  24]
 [ 28  35  36 482]]
tuning parameters: {'n_estimators': 90, 'max_features': 2}
ten most important keywords:
 ['windows', 'car', 'christian', 'god', 'government', 'team', 'jews', 'religion', 'space', 'graphics']


## 2. Build a boosting tree for this dataset 
and report the 5-fold cross validation value of the 
misclassification error. Similarly, report the best CV error,  the corresponding confusion matrix and 
tuning parameters. 

In [12]:
Y0 = Y-1
Y0test = Ytest-1

booster = GridSearchCV(
    xgb.XGBClassifier(max_depth=5, booster='gbtree', objective='multi:softmax'),
    param_grid={'n_estimators': np.arange(20, 100, 10), 'learning_rate': (0.1, 0.01)},
    cv=5)
booster.fit(X, Y0)
print(booster.best_params_)

{'learning_rate': 0.1, 'n_estimators': 90}


In [24]:
booster_best=booster.best_estimator_
print("best CV error:",CVError(booster_best,X,Y0,cv=5)) 
print("confusion matrix:\n",confusion_matrix(booster_best.predict(Xtest),Y0test))
print("tuning parameters:",booster.best_params_)

best CV error: 0.18738370571995389
confusion matrix:
 [[428  31  61  33]
 [  6 250   8   9]
 [ 21  15 145  32]
 [ 26  46  41 473]]
tuning parameters: {'learning_rate': 0.1, 'n_estimators': 90}


## 3. Compare the results from random forest and boosting trees. 

The result of the best CV error of random forest and boosting trees is quite similar. The parameter of n_estimators(the number of trees) is similar based on the limitation from 10 to 100. However, the boostring trees is more difficult to train, since you should tune the learning rate and max_depth by youself and it takes more time to train than random forest. Both of them consume much time.

## 4. Build a multi-class LDA classifier. 
Report the 5-fold CV error of misclassification and the confusion 
matrix. 

In [32]:
lda = LDA()
lda.fit(X, Y)

print("5-fold CV error:",CVError(lda,X,Y,cv=5))
print("confusion matrix:\n",confusion_matrix(lda.predict(Xtest),Ytest))

5-fold CV error: 0.20058739755877364
confusion matrix:
 [[416  35  55  34]
 [  3 238   9  13]
 [ 32  24 145  37]
 [ 30  45  46 463]]


## 5. Build a multi-class QDA classifier. 
Report the 5-fold CV error of misclassification and the confusion 
matrix.

In [33]:
qda = QDA()
qda.fit(X, Y)

print("5-fold CV error:",CVError(qda,X,Y,cv=5))
print("confusion matrix:\n",confusion_matrix(qda.predict(Xtest),Ytest))

5-fold CV error: 0.3147670276728789
confusion matrix:
 [[378  14  59  37]
 [ 86 321  95 215]
 [ 14   3  78  12]
 [  3   4  23 283]]


## 6. Compare the performances of all above methods and give your comments.

### Comparison of all above methods in table

| Model         | Parameters to tune                  | CV Error            |
| ------------- | ----------------------------------- | ------------------- |
| Random Forest | num of trees, num of features       | 0.18717899877054156 |
| Boosting Tree | num of trees, learning rates, depth | 0.18738370571995389 |
| LDA           |                                     | 0.20058739755877364 |
| QDA           |                                     | 0.3147670276728789  |

### Comments

The Random Forest and Boosting Tree models have better performance(based on CV Error metric), but there is not free launch since these two models require tuning parameter manually and massive computing time. LDA and QDA models run faster than previous two tree based model, but they also have limitation on the type of dataset. In this dataset, LDA performs better than QDA. However, in practice, we also have to try different methods and choose the best one.

# Part II. Spectral Clustering on 20newsgroup Data

In [53]:
def mis_clustering_error_rate(pred,real,nclass):
    pred_list = list(pred)
    errors = []
    for perm in permutations(range(nclass)):
        perm_pred = np.array([perm[x] for x in pred_list])
        errors.append(1-accuracy_score(perm_pred,real))
    return min(errors)

## 1. Apply PCA on the binary occurrence matrix and apply K-means clustering. 
Basically, take the top 
4 left singular vectors of the occurrence matrix (of size 16242x100) and apply K-means on the 
rows of these singular vectors with K=4. Report the mis-clustering error rate.

In [54]:
n4data = PCA(n_components=4).fit_transform(X)   
km4 = KMeans(n_clusters=4,random_state=5054)
km4.fit(n4data)
print("mis-clustering_error_rate:", mis_clustering_error_rate(km4.labels_,Y-1,4))

mis-clustering_error_rate: 0.591092563453513


## 2. Now take the top 5 left singular vectors of the occurrence matrix and apply K-means on the rows 
of these singular vectors with K=4. 
Report the mis-clustering error rate. 

In [55]:
n5data = PCA(n_components=5).fit_transform(X)   
km5 = KMeans(n_clusters=4,random_state=5054)
km5.fit(n4data)
print("mis-clustering_error_rate:", mis_clustering_error_rate(km5.labels_,Y-1,4))

mis-clustering_error_rate: 0.591092563453513


## 3. Compare with the performances from part I. 

The performance of this K-means based on PCA model is poorer than the model from part I, but it's easy to understand since these model in part II only use 4 or 5 principle components then the models can only use part of information from the dataset. However, on the other hand, this method is easier to perform and save computation time.